# Libraryes 
Download all the library needed to execution of the notebook

In [153]:
import pandas as pd
import numpy as np
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB.PDBParser import PDBParser
import matplotlib.pyplot as plt
import os
from Bio.PDB import PDBList, calc_angle, calc_dihedral, PPBuilder, is_aa, PDBIO, NeighborSearch,Selection
from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData
from Bio.PDB.PDBIO import Select
from Bio.SeqIO.PdbIO import PdbSeqresIterator
import numpy as np
import math
import matplotlib
import matplotlib.colors as mplcolors
import matplotlib.pyplot as plt
from RamachanDraw import phi_psi
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import copy

# Protein download form PDB
This is the protein to analyze '2lea_A' download the assigned PDB structure and consider only standard (non-hetero) residues of the specified chain (<PDB ID>_<chain ID>). 

Calculate the contact map (question 1) and the conformational energy (questions 2 and 3) as described in the IUPRED paper. 

The M and P matrices are available from the iupred_data.py. The smoothed energy is the moving average of the raw energy over a window of 21 residues (±10 residues around the current position).

In [154]:
path = "./home/Documents/Structural-Bioinformatics"
id_protein, chain = ['2lea', 'A' ]
parser = PDBParser(QUIET=True)
# if the file does not exist, it will be downloaded
if os.path.exists(path + "/pdb/pdb{}.ent".format(id_protein.lower())) is True:
  path_pdb = path + "/pdb/pdb{}.ent".format(id_protein.lower())
else:
  pdbl = PDBList()
  path_pdb = pdbl.retrieve_pdb_file(id_protein, pdir=path + "/pdb", file_format="pdb")
structure = parser.get_structure(id_protein, path_pdb)
new_structure = structure.copy()

if os.path.exists(path +'/figures_mid_term2') is False:
  os.mkdir(path +'/figures_mid_term2') 

# Defain main function to use

In [155]:
def get_distance_matrix(atoms, seq_sep=6):
  """
  Calculate a distance matrix considering only 
  CA atoms and a minimum sequence separation.  
  Return a Numpy array
  """
  distances = []
  for atom1 in atoms:
    row = []
    for atom2 in atoms:
      # Check sequence separation
      # if abs(atom1.id[1] - atom2.id[1]) >= seq_sep:
      if abs(atoms.index(atom1) - atoms.index(atom2)) >= seq_sep:
        row.append(atom1 - atom2)
      else:
        row.append(None) # For atoms not respecting sequence separation
    distances.append(row)
  return np.array(distances, dtype=float)

In [167]:
def get_contact_map(atoms, threshold=3.5, seq_sep=2):

  # Calculate contacts
  ns = NeighborSearch(atoms)

  # Init a empty matrix
  contact_map_nb = np.zeros((len(atoms), len(atoms)))

  # "search_all" returns the list of atoms in contact based on a distance cutoff
  # level="R" returns pairs of residues instead of atoms
  for residues1, residues2 in ns.search_all(threshold, 'R'):
    atoms_res1 = residues1.get_atoms()
    atoms_res2 = residues2.get_atoms()
    for atom1 in atoms_res1:
      for atom2 in atoms_res2:
        if abs(atoms.index(atom1) - atoms.index(atom2)) >= seq_sep:
          contact_map_nb[residues1.index(atom1), atoms.index(atom2)] = 1
          contact_map_nb[atoms.index(atom2), atoms.index(atom1)] = 1
  return contact_map_nb

#### Load the structure

In [157]:
pdbl = PDBList()
pdbl.retrieve_pdb_file(id_protein, pdir=path + '/pdbs', file_format='pdb')
structure = PDBParser(QUIET=True).get_structure(id_protein, path + "/pdbs/pdb{}.ent".format(id_protein))

Structure exists: './home/Documents/Structural-Bioinformatics/pdbs/pdb2lea.ent' 


#### Select the atoms

In [158]:
selected_atoms = [residue['CB'] if residue.has_id('CB') else residue['CA'] for residue in structure[0][chain] if residue.id[0] == " "]

#### Contacts calculated with Neighbours Search

In [168]:
contact_map_nb = get_contact_map(selected_atoms, threshold=3.5, seq_sep=2)

ValueError: <Atom N> is not in list